In [ ]:
!pip install spektral

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

from spektral import layers
from spektral.layers import GCNConv, MinCutPool, GlobalSumPool
from spektral.utils import label_to_one_hot
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
import os

In [ ]:
# Extra things for TensorBoard
import datetime
# Clear any logs from previous runs
!rm -rf ./logs/
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
################################################################################
# PARAMETERS
################################################################################
learning_rate = 1e-3  # Learning rate
epochs = 100          # Number of training epochs
batch_size = 16     # Batch size

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
################################################################################
# LOAD DATA
################################################################################
adj_matrix = np.load('/content/drive/MyDrive/XENON/GCN/wtdAdjMatrix.npy')
X = np.load('/content/drive/MyDrive/XENON/QM9/nodeFeatures.npy')
y = np.load('/content/drive/MyDrive/XENON/QM9/labels.npy')

In [ ]:
print(adj_matrix.shape)
print(X.shape)
print(y.shape)

(127, 127)
(97917, 127, 50)
(97917, 2)


In [ ]:
# Parameters
N = X.shape[-2]       # Number of nodes in the graphs
F = X[0].shape[-1]    # Dimension of node features
n_out = y.shape[-1]   # Dimension of the target

In [ ]:
# Train/test split
X_train, X_test, \
y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Train/test split
X_val, X_test, \
y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=0)

In [ ]:
del X, y

In [ ]:
%tensorboard --logdir logs/fit

In [ ]:
# Input/GCN(1)→GCN(16)→GCN(32)→Flatten→Dense(8)→Dense(2)/output


X_in = Input(shape=(N, F))
A_in = Input(shape=(N, N))

X_1 = GCNConv(16, activation='relu')([X_in, A_in])
X_1, A_1 = MinCutPool(N // 2)([X_1, A_in])
X_2 = GCNConv(32, activation='relu')([X_1, A_1])
X_3 = GlobalSumPool()(X_2)
X_4 = Dense(8)(X_3)
output = Dense(n_out)(X_4)


# Build model
model = Model(inputs=[X_in, A_in], outputs=output)
optimizer = Adam(lr=learning_rate)
model.compile(optimizer=optimizer, loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError()])
model.summary()

Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            [(None, 127, 50)]    0                                            
__________________________________________________________________________________________________
input_10 (InputLayer)           [(None, 127, 127)]   0                                            
__________________________________________________________________________________________________
gcn_conv_7 (GCNConv)            (None, 127, 16)      800         input_9[0][0]                    
                                                                 input_10[0][0]                   
__________________________________________________________________________________________________
min_cut_pool_3 (MinCutPool)     [(None, 63, 16), (No 1071        gcn_conv_7[0][0]      

In [ ]:
import random

def gen(features, labels, batch_size, pmt_count, res_time):
 # Create empty arrays to contain batch of features and labels#
 batch_features = np.zeros((batch_size, pmt_count, res_time))
 batch_labels = np.zeros((batch_size, 1))
 A = np.repeat(adj_matrix[np.newaxis,...], batch_size, axis=0)
 while True:
   for i in range(batch_size):
     # choose random index in features
     index = random.choices(range(len(labels)),k=batch_size)
     batch_features = features[index]
     batch_labels = labels[index]
   yield [batch_features, A], batch_labels

In [ ]:
if os.path.isdir('/content/drive/MyDrive/XENON/GCN/ckpts_C')==False:
  os.mkdir('/content/drive/MyDrive/XENON/GCN/ckpts_C')

In [ ]:
import os
from tensorflow.keras.callbacks import ModelCheckpoint

path="/content/drive/MyDrive/XENON/GCN/ckpts_C/"+"cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(path)

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# Create a callback that saves the model's weights 
cp_callback = ModelCheckpoint(
    filepath=path, 
    verbose=1, 
    save_weights_only=True,
    monitor='loss', 
    save_best_only=True)

callbacks_list=[tensorboard_callback, cp_callback]

In [ ]:
history=model.fit_generator(gen(X_train, y_train, batch_size, N, F), 
                    steps_per_epoch=y_train.shape[0]//batch_size, 
                    epochs=75, 
                    validation_data=gen(X_val, y_val, batch_size, N, F),
                    validation_steps=y_val.shape[0]//batch_size,
                    callbacks=callbacks_list
                    )

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/75
   1/4895 [..............................] - ETA: 0s - loss: 703.6055 - root_mean_squared_error: 26.5195WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
4894/4895 [============================>.] - ETA: 0s - loss: 626.2761 - root_mean_squared_error: 25.0191
Epoch 00001: loss improved from inf to 626.26190, saving model to /content/drive/MyDrive/XENON/GCN/ckpts_C/cp-0001.ckpt
4895/4895 [==============================] - 203s 41ms/step - loss: 626.2619 - root_mean_squared_error: 25.0188 - val_loss: 626.4420 - val_root_mean_squared_error: 25.0224
Epoch 2/75
 875/4895 [====>.........................] - ETA: 3:08 - loss: 624.6455 - root_mean_squared_error: 24.9865

KeyboardInterrupt: ignored

In [ ]:
model.save("/content/drive/MyDrive/XENON/GCN/modelC")

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /content/drive/MyDrive/XENON/GCN/modelA/assets
